# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\\$645 - 675) underperformed compared to schools with smaller budgets (\\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

**Note:**
Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [168]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import sqlite3
import os

current_directory = os.getcwd()
print(current_directory)

# loading data from python.db database
con = sqlite3.connect('python.db')

# import 'school' and 'student' tables into pandas dataframe
school_data = pd.read_sql('SELECT * FROM school', con)
student_data = pd.read_sql('SELECT * FROM student', con)

C:\Users\patron\Desktop\Python


In [169]:
# join the two tables into a single dataframe
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

,id_x,Student ID,student_name,gender,grade,school_name,reading_score,math_score,id_y,School ID,type,size,budget
0,0,0,Paul Bradley,M,9th,Huang High School,66,79,0,0,District,2917,1910635
1,1,1,Victor Smith,M,12th,Huang High School,94,61,0,0,District,2917,1910635
2,2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,0,District,2917,1910635
3,3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,0,District,2917,1910635
4,4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,0,District,2917,1910635


## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [173]:
# Create a District Summary

district_schools = school_data_complete[school_data_complete['type'] == 'District']
district_schools.head(10)

# Create Summary Datafram for District 
district_summary = pd.DataFrame({
    "Total Schools": [total_district_schools],
    "Total Students": [total_district_students],
    "Total Budget": [f"${total_district_budget:,.2f}"],
    "Average Math Score": [round(average_math_score, 2)],
    "Average Reading Score": [round(average_reading_score, 2)],
    "Overall Average Score": [round(overall_average_score, 2)],
    "Passing Math (%)": [f"{passing_math_percentage:.2f}%"],
    "Passing Reading (%)": [f"{passing_reading_percentage:.2f}%"]
})

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,Overall Average Score,Passing Math (%),Passing Reading (%)
0,7,26976,"$17,347,923.00",76.99,80.96,78.97,66.52%,80.91%


In [174]:
# Total number of schools

total_district_schools = district_schools['school_name'].nunique()
print("Total number of district schools:", total_district_schools)



Total number of district schools: 7


In [177]:
# Total number of students

total_district_students = district_schools.shape[0]
print("Total number of students in district schools:", total_district_students)




Total number of students in district schools: 26976


In [178]:
# Total budget

# Get unique district schools
unique_district_schools = district_schools[['school_name', 'budget']].drop_duplicates()

# Calculate total budget
total_district_budget = unique_district_schools['budget'].sum()
formatted_budget = f"${total_district_budget:,.2f}"
print("Total budget for the 7 district schools:", formatted_budget)





Total budget for the 7 district schools: $17,347,923.00


In [181]:
# Average math score

average_math_score = district_schools['math_score'].mean()
print(f"Average math score for students in district schools: {average_math_score:.2f}")



Average math score for students in district schools: 76.99


In [182]:
# Average reading score

average_reading_score = district_schools['reading_score'].mean()
print(f"Average reading score for students in district schools: {average_reading_score:.2f}")



Average reading score for students in district schools: 80.96


In [185]:
# Overall average score

overall_average_score = (average_reading_score + average_math_score) / 2
print(f"Overall average score (reading & math) for students in district schools: {overall_average_score:.2f}")


Overall average score (reading & math) for students in district schools: 78.97


In [187]:
# Percentage of passing math (70 or greater)

# Count students with math_score >= 70
passing_math = district_schools[district_schools['math_score'] >= 70].shape[0]
passing_math_percentage = (passing_math / total_district_students) * 100
print(f"Percentage of students passing math in the district: {passing_math_percentage:.2f}%")


# Percentage of passing reading (70 or greater)

# Count students with reading_score >= 70
passing_reading = district_schools[district_schools['reading_score'] >= 70].shape[0]
passing_reading_percentage = (passing_reading / total_district_students) * 100
print(f"Percentage of students passing reading: {passing_reading_percentage:.2f}%")



Percentage of students passing math in the district: 66.52%
Percentage of students passing reading: 80.91%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

### Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [193]:
#  Sort and display the top five schools in overall passing rate

# Sort schools by overall passing rate in descending order
top_5_schools = overall_pass_rate.sort_values(by='Overall Passing Rate (%)', ascending=False).head(5)

# Display the result
print(top_5_schools)



            School Name  Overall Passing Rate (%)
1   Cabrera High School                 95.586652
12   Thomas High School                 95.290520
9      Pena High School                 95.270270
4   Griffin High School                 95.265668
13   Wilson High School                 95.203679


In [195]:
# Calculate total school budget

# Group by school and get total school budget
school_budget = school_data.groupby('school_name')['budget'].sum().reset_index()

# Rename columns for clarity
school_budget.columns = ['School Name', 'Total School Budget']

# Display the result
print(school_budget)



              School Name  Total School Budget
0      Bailey High School              3124928
1     Cabrera High School              1081356
2    Figueroa High School              1884411
3        Ford High School              1763916
4     Griffin High School               917500
5   Hernandez High School              3022020
6      Holden High School               248087
7       Huang High School              1910635
8     Johnson High School              3094650
9        Pena High School               585858
10  Rodriguez High School              2547363
11    Shelton High School              1056600
12     Thomas High School              1043130
13     Wilson High School              1319574
14     Wright High School              1049400


In [197]:
# Calculate per student budget

# Count number of students per school
total_students_per_school = school_data_complete.groupby('school_name')['student_name'].count().reset_index()

# Rename columns for clarity
total_students_per_school.columns = ['School Name', 'Total Students']

# Display the result
print(total_students_per_school)

# Merge total students and school budget on School Name
school_budget_per_student = pd.merge(school_budget, total_students_per_school, on='School Name')

# Calculate per student budget
school_budget_per_student['Per Student Budget'] = (
    school_budget_per_student['Total School Budget'] / school_budget_per_student['Total Students']
)

# Reorder columns for clarity
school_budget_per_student = school_budget_per_student[
    ['School Name', 'Total School Budget', 'Total Students', 'Per Student Budget']
]

# Display the result
print(school_budget_per_student)




              School Name  Total Students
0      Bailey High School            4976
1     Cabrera High School            1858
2    Figueroa High School            2949
3        Ford High School            2739
4     Griffin High School            1468
5   Hernandez High School            4635
6      Holden High School             427
7       Huang High School            2917
8     Johnson High School            4761
9        Pena High School             962
10  Rodriguez High School            3999
11    Shelton High School            1761
12     Thomas High School            1635
13     Wilson High School            2283
14     Wright High School            1800
              School Name  Total School Budget  Total Students  \
0      Bailey High School              3124928            4976   
1     Cabrera High School              1081356            1858   
2    Figueroa High School              1884411            2949   
3        Ford High School              1763916            2739  

In [199]:
# Cacluate the avg math and reading score

# Calculate average math score for each school
average_math_scores_by_school = school_data_complete.groupby('school_name')['math_score'].mean().reset_index()

# Rename columns for clarity
average_math_scores_by_school.columns = ['School Name', 'Average Math Score']

# Display the result
print(average_math_scores_by_school)

# Calculate average reading score for each school
average_reading_scores_by_school = school_data_complete.groupby('school_name')['reading_score'].mean().reset_index()

# Rename columns for clarity
average_reading_scores_by_school.columns = ['School Name', 'Average Reading Score']

# Display the result
print(average_reading_scores_by_school)




              School Name  Average Math Score
0      Bailey High School           77.048432
1     Cabrera High School           83.061895
2    Figueroa High School           76.711767
3        Ford High School           77.102592
4     Griffin High School           83.351499
5   Hernandez High School           77.289752
6      Holden High School           83.803279
7       Huang High School           76.629414
8     Johnson High School           77.072464
9        Pena High School           83.839917
10  Rodriguez High School           76.842711
11    Shelton High School           83.359455
12     Thomas High School           83.418349
13     Wilson High School           83.274201
14     Wright High School           83.682222
              School Name  Average Reading Score
0      Bailey High School              81.033963
1     Cabrera High School              83.975780
2    Figueroa High School              81.158020
3        Ford High School              80.746258
4     Griffin High 

#### Find the passing rate for math and reading (above 70 points)

In [202]:
# Find the total counts of math result

# Count of student math scores per school
math_result_counts = school_data_complete.groupby('school_name')['math_score'].count().reset_index()

# Rename columns for clarity
math_result_counts.columns = ['School Name', 'Total Math Results']

# Display the result
print(math_result_counts)


# Find the counts for math result in each school that pass 70 or higher

# Filter for passing math scores (70 or higher)
passing_math = school_data_complete[school_data_complete['math_score'] >= 70]

# Count passing students per school
passing_math_counts = passing_math.groupby('school_name')['math_score'].count().reset_index()

# Rename columns for clarity
passing_math_counts.columns = ['School Name', 'Passing Math Count']

# Display the result
print(passing_math_counts)


# Calculate the math passing rate

# Merge passing and total counts on school name
math_pass_rate = pd.merge(passing_math_counts, math_result_counts, on='School Name')

# Calculate passing rate
math_pass_rate['Math Passing Rate (%)'] = (math_pass_rate['Passing Math Count'] / math_pass_rate['Total Math Results']) * 100

# Reorder columns to show School Name first
math_pass_rate = math_pass_rate[['School Name', 'Passing Math Count', 'Total Math Results', 'Math Passing Rate (%)']]

# Display the result
print(math_pass_rate)





              School Name  Total Math Results
0      Bailey High School                4976
1     Cabrera High School                1858
2    Figueroa High School                2949
3        Ford High School                2739
4     Griffin High School                1468
5   Hernandez High School                4635
6      Holden High School                 427
7       Huang High School                2917
8     Johnson High School                4761
9        Pena High School                 962
10  Rodriguez High School                3999
11    Shelton High School                1761
12     Thomas High School                1635
13     Wilson High School                2283
14     Wright High School                1800
              School Name  Passing Math Count
0      Bailey High School                3318
1     Cabrera High School                1749
2    Figueroa High School                1946
3        Ford High School                1871
4     Griffin High School         

In [204]:
# Find the total counts of read result

# Count of student reading scores per school
reading_result_counts = school_data_complete.groupby('school_name')['reading_score'].count().reset_index()

# Rename columns for clarity
reading_result_counts.columns = ['School Name', 'Total Reading Results']

# Display the result
print(reading_result_counts)


# Find the counts for read result in each school that pass 70 or higher

# Filter for passing reading scores (70 or higher)
passing_reading = school_data_complete[school_data_complete['reading_score'] >= 70]

# Count passing students per school
passing_reading_counts = passing_reading.groupby('school_name')['reading_score'].count().reset_index()

# Rename columns for clarity
passing_reading_counts.columns = ['School Name', 'Passing Reading Count']

# Display the result
print(passing_reading_counts)


# Calculate the read passing rate

# Merge total reading results with passing reading counts
reading_pass_rate = pd.merge(passing_reading_counts, reading_result_counts, on='School Name')

# Calculate reading passing rate
reading_pass_rate['Reading Passing Rate (%)'] = (
    reading_pass_rate['Passing Reading Count'] / reading_pass_rate['Total Reading Results']
) * 100

# Reorder columns for clarity
reading_pass_rate = reading_pass_rate[
    ['School Name', 'Passing Reading Count', 'Total Reading Results', 'Reading Passing Rate (%)']
]

# Display the result
print(reading_pass_rate)


              School Name  Total Reading Results
0      Bailey High School                   4976
1     Cabrera High School                   1858
2    Figueroa High School                   2949
3        Ford High School                   2739
4     Griffin High School                   1468
5   Hernandez High School                   4635
6      Holden High School                    427
7       Huang High School                   2917
8     Johnson High School                   4761
9        Pena High School                    962
10  Rodriguez High School                   3999
11    Shelton High School                   1761
12     Thomas High School                   1635
13     Wilson High School                   2283
14     Wright High School                   1800
              School Name  Passing Reading Count
0      Bailey High School                   4077
1     Cabrera High School                   1803
2    Figueroa High School                   2381
3        Ford High S

In [206]:
# Calculate the overall passing rate (average of the math and reading passing rate)

# Merge math and reading passing rates on School Name
overall_pass_rate = pd.merge(math_pass_rate, reading_pass_rate, on='School Name')

# Calculate overall passing rate
overall_pass_rate['Overall Passing Rate (%)'] = (
    (overall_pass_rate['Math Passing Rate (%)'] + overall_pass_rate['Reading Passing Rate (%)']) / 2
)

# Keep only relevant columns for clarity
overall_pass_rate = overall_pass_rate[[
    'School Name',
    'Overall Passing Rate (%)'
]]

# Display the result
print(overall_pass_rate)



              School Name  Overall Passing Rate (%)
0      Bailey High School                 74.306672
1     Cabrera High School                 95.586652
2    Figueroa High School                 73.363852
3        Ford High School                 73.804308
4     Griffin High School                 95.265668
5   Hernandez High School                 73.807983
6      Holden High School                 94.379391
7       Huang High School                 73.500171
8     Johnson High School                 73.639992
9        Pena High School                 95.270270
10  Rodriguez High School                 73.293323
11    Shelton High School                 94.860875
12     Thomas High School                 95.290520
13     Wilson High School                 95.203679
14     Wright High School                 94.972222


In [208]:
# Merge all relevant dataframes on School Name
overview = school_budget.merge(
    total_students_per_school, on='School Name'
).merge(
    school_budget_per_student[['School Name', 'Per Student Budget']], on='School Name'
).merge(
    average_math_scores_by_school, on='School Name'
).merge(
    average_reading_scores_by_school, on='School Name'
).merge(
    math_pass_rate[['School Name', 'Math Passing Rate (%)']], on='School Name'
).merge(
    reading_pass_rate[['School Name', 'Reading Passing Rate (%)']], on='School Name'
).merge(
    overall_pass_rate[['School Name', 'Overall Passing Rate (%)']], on='School Name'
).merge(
    school_data[['school_name', 'type']].drop_duplicates(), left_on='School Name', right_on='school_name'
)

# Rename and reorder columns
overview.rename(columns={
    'type': 'School Type'
}, inplace=True)

overview = overview[[
    'School Name', 'School Type', 'Total Students', 'Total School Budget', 'Per Student Budget',
    'Average Math Score', 'Average Reading Score',
    'Math Passing Rate (%)', 'Reading Passing Rate (%)', 'Overall Passing Rate (%)'
]]

# Display the final overview dataframe
print(overview)

              School Name School Type  Total Students  Total School Budget  \
0      Bailey High School    District            4976              3124928   
1     Cabrera High School     Charter            1858              1081356   
2    Figueroa High School    District            2949              1884411   
3        Ford High School    District            2739              1763916   
4     Griffin High School     Charter            1468               917500   
5   Hernandez High School    District            4635              3022020   
6      Holden High School     Charter             427               248087   
7       Huang High School    District            2917              1910635   
8     Johnson High School    District            4761              3094650   
9        Pena High School     Charter             962               585858   
10  Rodriguez High School    District            3999              2547363   
11    Shelton High School     Charter            1761           

### Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [214]:
#  Sort and display the worst five schools in overall passing rate

# Sort schools by overall passing rate in ascending order using overall_pass_rate
worst_schools = overall_pass_rate.sort_values(by='Overall Passing Rate (%)', ascending=True).head(5)

# Display the result
print(worst_schools)



              School Name  Overall Passing Rate (%)
10  Rodriguez High School                 73.293323
2    Figueroa High School                 73.363852
7       Huang High School                 73.500171
8     Johnson High School                 73.639992
3        Ford High School                 73.804308


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [233]:
# Create table that lists the average math score for each school of each grade level.

# Combine into one DataFrame
avg_math_by_grade = pd.DataFrame({
    '9th Grade': avg_math_9,
    '10th Grade': avg_math_10,
    '11th Grade': avg_math_11,
    '12th Grade': avg_math_12
})

# Display result
avg_math_by_grade


,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,


In [235]:
# Calculate the average math score for 9th grade in each school

grade_9 = school_data_complete[school_data_complete['grade'] == '9th']
avg_math_9 = grade_9.groupby('school_name')['math_score'].mean()


In [237]:
# Calculate the average math score for 10th grade in each school

grade_10 = school_data_complete[school_data_complete['grade'] == '10th']
avg_math_10 = grade_10.groupby('school_name')['math_score'].mean()


In [239]:
# Calculate the average math score for 11th grade in each school

grade_11 = school_data_complete[school_data_complete['grade'] == '11th']
avg_math_11 = grade_11.groupby('school_name')['math_score'].mean()


In [243]:
# Calculate the average math score for 12th grade in each school

grade_12 = school_data_complete[school_data_complete['grade'] == '12th']
avg_math_12 = grade_12.groupby('school_name')['math_score'].mean()

# Combine into one DataFrame
avg_math_by_grade = pd.DataFrame({
    '9th Grade': avg_math_9,
    '10th Grade': avg_math_10,
    '11th Grade': avg_math_11,
    '12th Grade': avg_math_12
})

# Display result
avg_math_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


### Reading Score by Grade 

* Perform the same operations as above for reading scores

In [259]:
# Create table that lists the average reading score for each school of each grade level.

avg_reading_by_grade = pd.DataFrame({
    '9th Grade': avg_reading_9,
    '10th Grade': avg_reading_10,
    '11th Grade': avg_reading_11,
    '12th Grade': avg_reading_12
})

# Display result
avg_reading_by_grade

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


In [261]:
# Calculate the average reading score for 9th grade in each school

# Filter by grade (strings like '9th', '10th', etc.)
grade_9 = school_data_complete[school_data_complete['grade'] == '9th']
avg_reading_9 = grade_9.groupby('school_name')['reading_score'].mean()

grade_10 = school_data_complete[school_data_complete['grade'] == '10th']
avg_reading_10 = grade_10.groupby('school_name')['reading_score'].mean()

grade_11 = school_data_complete[school_data_complete['grade'] == '11th']
avg_reading_11 = grade_11.groupby('school_name')['reading_score'].mean()

grade_12 = school_data_complete[school_data_complete['grade'] == '12th']
avg_reading_12 = grade_12.groupby('school_name')['reading_score'].mean()



In [263]:
# Calculate the average reading score for 10th grade in each school

grade_10 = school_data_complete[school_data_complete['grade'] == '10th']
avg_reading_10 = grade_10.groupby('school_name')['reading_score'].mean()


In [265]:
# Calculate the average reading score for 11th grade in each school

grade_11 = school_data_complete[school_data_complete['grade'] == '11th']
avg_reading_11 = grade_11.groupby('school_name')['reading_score'].mean()


In [267]:
# Calculate the average reading score for 12th grade in each school

grade_12 = school_data_complete[school_data_complete['grade'] == '12th']
avg_reading_12 = grade_12.groupby('school_name')['reading_score'].mean()

# Combine into one DataFrame
avg_reading_by_grade = pd.DataFrame({
    '9th Grade': avg_reading_9,
    '10th Grade': avg_reading_10,
    '11th Grade': avg_reading_11,
    '12th Grade': avg_reading_12
})

# Display result
avg_reading_by_grade



,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [269]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$616-645", "$646-675"]

In [271]:
# Create a new column to show budget per student in each row

school_data_complete["Per Student Budget"] = (
    school_data_complete["budget"] / school_data_complete["size"]
)



In [273]:
# Create a new column to define the spending ranges per student

# Create the spending range column using pd.cut()
school_data_complete["Spending Ranges (Per Student)"] = pd.cut(
    school_data_complete["Per Student Budget"],
    bins=spending_bins,
    labels=group_names,
    include_lowest=True,
    right=False  # left-inclusive, prevents overlap
)



In [275]:
# Calculate the average math score within each spending range

average_math_by_spending = school_data_complete.groupby(
    "Spending Ranges (Per Student)"
)["math_score"].mean()



C:\Users\patron\AppData\Local\Temp\ipykernel_26776\789165630.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  average_math_by_spending = school_data_complete.groupby(


In [277]:
# Calculate the percentage passing rate for math in each spending range

passing_math_by_spending = school_data_complete.groupby(
    "Spending Ranges (Per Student)"
)["math_score"].apply(lambda x: (x >= 70).mean())
print((passing_math_by_spending * 100).round(2))




Spending Ranges (Per Student)
<$585       93.70
$585-615    94.12
$616-645    71.40
$646-675    66.23
Name: math_score, dtype: float64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\3092641860.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math_by_spending = school_data_complete.groupby(


In [279]:
# Calculate the percentage passing rate for reading in each spending range

passing_reading_by_spending = school_data_complete.groupby(
    "Spending Ranges (Per Student)"
)["reading_score"].apply(lambda x: (x >= 70).mean())
print((passing_reading_by_spending * 100).round(2))



Spending Ranges (Per Student)
<$585       96.69
$585-615    95.89
$616-645    83.61
$646-675    81.11
Name: reading_score, dtype: float64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\2841302235.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_reading_by_spending = school_data_complete.groupby(


In [281]:
# Calculate the percentage overall passing rate in each spending range

# Calculate passing rates
passing_math = school_data_complete.groupby("Spending Ranges (Per Student)")["math_score"].apply(lambda x: (x >= 70).mean())
passing_reading = school_data_complete.groupby("Spending Ranges (Per Student)")["reading_score"].apply(lambda x: (x >= 70).mean())

# Calculate overall passing rate as the average of the two
overall_passing_rate = (passing_math + passing_reading) / 2

# Display as percentage
print((overall_passing_rate * 100).round(2))



Spending Ranges (Per Student)
<$585       95.19
$585-615    95.01
$616-645    77.51
$646-675    73.67
dtype: float64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\2165097707.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_math = school_data_complete.groupby("Spending Ranges (Per Student)")["math_score"].apply(lambda x: (x >= 70).mean())
C:\Users\patron\AppData\Local\Temp\ipykernel_26776\2165097707.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  passing_reading = school_data_complete.groupby("Spending Ranges (Per Student)")["reading_score"].apply(lambda x: (x >= 70).mean())


### Scores by School Size

* Perform the same operations as above, based on school size.

In [283]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [285]:
# Create a new column for the bin groups

school_data_complete["School Size"] = pd.cut(
    school_data_complete["size"],
    bins=size_bins,
    labels=group_names,
    include_lowest=True,
    right=False 
)

Look for the total count of test scores that pass 70% or higher




In [288]:
# math_pass_size

# Count of students with math scores >= 70 in each school size group
math_pass_size = school_data_complete[school_data_complete["math_score"] >= 70].groupby("School Size")["math_score"].count()

print(math_pass_size)


School Size
Small (<1000)          1305
Medium (1000-2000)     7978
Large (2000-5000)     20087
Name: math_score, dtype: int64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\2536240649.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  math_pass_size = school_data_complete[school_data_complete["math_score"] >= 70].groupby("School Size")["math_score"].count()


In [290]:
# read_pass_size

# Count of students with reading scores >= 70 in each school size group
read_pass_size = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("School Size")["reading_score"].count()

print(read_pass_size)



School Size
Small (<1000)          1334
Medium (1000-2000)     8247
Large (2000-5000)     24029
Name: reading_score, dtype: int64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\2581597528.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  read_pass_size = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("School Size")["reading_score"].count()


In [292]:
# Calculate the overall passing rate for different school size

# Create a column indicating whether a student passed both math and reading
school_data_complete["overall_pass"] = (
    (school_data_complete["math_score"] >= 70) & 
    (school_data_complete["reading_score"] >= 70)
)

# Group by school size and calculate the overall passing rate
overall_passing_rate_by_size = school_data_complete.groupby("School Size")["overall_pass"].mean()

# Optional: Display as percentages
print((overall_passing_rate_by_size * 100).round(2))



School Size
Small (<1000)         90.14
Medium (1000-2000)    90.62
Large (2000-5000)     56.57
Name: overall_pass, dtype: float64


C:\Users\patron\AppData\Local\Temp\ipykernel_26776\3093511922.py:10: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  overall_passing_rate_by_size = school_data_complete.groupby("School Size")["overall_pass"].mean()


### Scores by School Type

* Perform the same operations as above, based on school type.

In [294]:
# Create bins and groups, school type {'Charter', 'District'}

group_names = ["Charter", "District"]



Find counts of the passing 70 or higher score for the both test


In [297]:
# math pass size

# Count of students with math scores >= 70, grouped by school type
math_pass_type = school_data_complete[school_data_complete["math_score"] >= 70].groupby("type")["math_score"].count()

print(math_pass_type)



type
Charter     11426
District    17944
Name: math_score, dtype: int64


In [299]:
# reading pass size

# Count of students with reading scores >= 70, grouped by school type
read_pass_type = school_data_complete[school_data_complete["reading_score"] >= 70].groupby("type")["reading_score"].count()

print(read_pass_type)


type
Charter     11785
District    21825
Name: reading_score, dtype: int64


In [301]:
# Calculate the overall passing rate

# Create a column for students who passed both math and reading
school_data_complete["overall_pass"] = (
    (school_data_complete["math_score"] >= 70) &
    (school_data_complete["reading_score"] >= 70)
)

# Calculate the overall passing rate by school type
overall_passing_rate_by_type = school_data_complete.groupby("type")["overall_pass"].mean()

# Optional: Format as percentages
print((overall_passing_rate_by_type * 100).round(2))


type
Charter     90.56
District    53.70
Name: overall_pass, dtype: float64
